In [73]:

import pickle
dataset="ACM"
with open('data/'+dataset+'/node_features.pkl','rb') as f:
    node_features = pickle.load(f)
with open('data/'+dataset+'/edges.pkl','rb') as f:
    edges = pickle.load(f)
with open('data/'+dataset+'/labels.pkl','rb') as f:
    labels = pickle.load(f)

In [74]:
print(len(labels[0]))
print(len(labels[1]))
print(len(labels[2]))
#print(labels[0])

600
300
2125


In [75]:
edges[0].nonzero()[0]

array([   0,    0,    0, ..., 3024, 3024, 3024], dtype=int32)

In [76]:
num_nodes=node_features.shape[0]

In [77]:
edge_occur_dict={}
for i in range(num_nodes):
    edge_occur_dict[i]=0
for eadj in edges:
    elist=eadj.nonzero()
    for el in elist:
        for id in el:
            edge_occur_dict[id]=1
for k in edge_occur_dict:
    v=edge_occur_dict[k]
    if v==0:
        print(k)

#首先，确定没有孤立点

In [78]:
#然后，开始按照edge type 来还原node type
num_nodes=node_features.shape[0]
ntype_pairs_by_etype={}
for i in range(len(edges)):
    eadj=edges[i]
    elist=eadj.nonzero()
    ntype_pairs_by_etype[i]=[]
    for src_dst in elist:
        temp=[]
        for idx in src_dst:
            temp.append(idx)
        temp=list(set(temp))
        ntype_pairs_by_etype[i].append(temp)

#ntype_pairs_by_etype[0][0]


def compare_list(listA,listB,num_nodes):
    dA={}
    dB={}
    for i in range(num_nodes):
        dA[i]=0
        dB[i]=0
    for i in listA:
        dA[i]=1
    for i in listB:
        dB[i]=1
    #check
    for i in range(num_nodes):
        if dA[i]!=dB[i]:
            return False
    return True

ntype_count=0
ntype_indexer={}
for _ in ntype_pairs_by_etype.values():
    for idxset in _:
        idxset=list(set(idxset))
        #print(len(set(idxset)))
        FLAG_new=True
        for ntype in ntype_indexer.keys():
            x=list(set(ntype_indexer[ntype]))
            print(len(idxset),len(x),)
            if compare_list(x,idxset,num_nodes):
                FLAG_new=False
                continue
            if len(set(x).intersection(set(idxset)))>0:
                #如有相交则必包含
                #assert set(x).issubset(set(idxset)) or set(idxset).issubset(set(x))
                #合并为大的那一个
                if set(idxset).issubset(set(x)):
                    FLAG_new=False
                elif set(x).issubset(set(idxset)):
                    ntype_indexer[ntype]=set(idxset)
                    FLAG_new=False
                #如有相交则必包含
                else:
                    raise Exception("如有相交则必包含")

        if FLAG_new==True:
            ntype_indexer[ntype_count]=list(set(idxset))
            ntype_count+=1


#检查是否存在包含关系，如果相交则必包含






5912 3020
5912 3020
5912 5912
3020 3020
3020 5912
3025 3020
3025 5912
57 3025
57 5912
57 3025
57 5912
57 57
3025 3025
3025 5912
3025 57


In [79]:
#检查node type是否有唯一性
for i in range(len(ntype_indexer.keys())):
    for j in range(len(ntype_indexer.keys())):
        if i!=j:
            x=ntype_indexer[i]
            y=ntype_indexer[j]
            inter=set(x).intersection(set(y))
            print(f"intersection: {inter}",i,j)
                

intersection: set() 0 1
intersection: set() 0 2
intersection: set() 1 0
intersection: set() 1 2
intersection: set() 2 0
intersection: set() 2 1


In [80]:
#检查node type是否有合共性（放一起构成全集）
temp=[]
for x in ntype_indexer.values():
    temp.extend(x)
print(compare_list( temp,list(range(num_nodes))  ,num_nodes))

True


In [81]:
#检查是否需要id-remapping
id_remapping_FLAG=False
for idxset in ntype_indexer.values():
    idxset=sorted(idxset)
    for i in range(len(idxset)-1):
        if idxset[i]!=idxset[i+1]-1:     #node id 需要是连续的，在同一个node type里面
            print(f"idxset[i],idxset[i+1] : {idxset[i],idxset[i+1]}")
            id_remapping_FLAG=True


    



In [82]:
#做id-remapping
if id_remapping_FLAG:
    id_mapping={}
    count=0
    for idxset in ntype_indexer.values():
        print(f"idxset:{len(idxset)} set:{len(set(idxset))}")
        for idx in set(idxset):
            id_mapping[idx]=count
            count+=1

    assert num_nodes==  max(id_mapping.values())






    raise NotImplementedError("!!!!")


In [83]:
#给node id们type
sorted_ntype_indexer={}
for key in ntype_indexer.keys():
    sorted_ntype_indexer[key]=min(ntype_indexer[key])
order_mapping={}
sorted_ntype_indexer=sorted( [ (key,sorted_ntype_indexer[key]) for key in sorted_ntype_indexer.keys() ],  key=lambda x:x[1]   )
for i,item in enumerate(sorted_ntype_indexer):
    order_mapping[item[0]]=i
temp={}
for key in ntype_indexer.keys():
    temp[order_mapping[key]]=ntype_indexer[key]
ntype_indexer=temp
node_ntype={}
for i in range(num_nodes):
    node_ntype[i]=None
for key in ntype_indexer.keys():
    for node in ntype_indexer[key]:
        node_ntype[node]=key

In [84]:
#generate node.dat


with open("node.dat","w") as f:
    for i in range(num_nodes):
        idx=str(i)
        name=""
        ntype=str(node_ntype[i])
        feature=",".join(map(  lambda x:str(x) ,   list(node_features[i])  ))
        f.write(idx+"\t"+name+"\t"+ntype  +"\t"+feature+"\n")

In [ ]:
#generate link.dat

with open("link.dat","w") as f:
    for etype,eadj in enumerate(edges):
        elist=eadj.nonzero()
        src,dst=elist
        for i in range(len(src)):
            s,d=str(src[i]),str(dst[i])
            et=str(etype)
            weight=str(1)
            f.write(s+"\t"+d+"\t"+et  +"\t"+weight+"\n")

In [ ]:
#generate label.dat
train=list(labels[0])
valid=list(labels[1])
train.extend(valid)
test=list(labels[2])
with open("label.dat","w") as f:
    for idx,label in train:
        ntype=str(node_ntype[idx])
        idx=str(idx)
        name=""
        label=str(label)
        f.write(idx+"\t"+name+"\t"+ntype+"\t"+label+"\n")

In [ ]:
#generate label.dat
with open("label.dat.test","w") as f:
    for idx,label in test:
        ntype=str(node_ntype[idx])
        idx=str(idx)
        name=""
        label=str(label)
        f.write(idx+"\t"+name+"\t"+ntype+"\t"+label+"\n")